In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
spark = SparkSession.builder.appName('Spark DataFrame').getOrCreate()
df = spark.read.option('header',True).csv('/FileStore/tables/studentdata.csv')


In [0]:
df.count() #1000 rows present in the dataframe

Out[2]: 1000

In [0]:
#count of students who have enrolled for DB
df.filter(df.course=='DB').count()

Out[3]: 157

In [0]:
df.distinct().count() #all the rows are unique

Out[4]: 1000

In [0]:
df.select('gender', 'age').distinct().count() 

Out[6]: 4

In [0]:
#df.dropDuplicates(['gender']).show()

+---+------+---------------+------+-----+-----+--------------------+
|age|gender|           name|course| roll|marks|               email|
+---+------+---------------+------+-----+-----+--------------------+
| 28|Female|Hubert Oliveras|    DB|02984|   59|Annika Hoffman_Na...|
| 28|  Male| Celeste Lollis|    PF|21267|   45|Jeannetta Golden_...|
+---+------+---------------+------+-----+-----+--------------------+



In [0]:
df.dropDuplicates(['gender', 'course']).count()

Out[9]: 12

In [0]:
df2= df.dropDuplicates(['age', 'gender','course'])
#df2.show()

In [0]:
#sorting based on the column
#df.sort("marks","age").show()

In [0]:
#df.sort(df.marks.asc(), df.age.desc()).show()
#df.orderBy(df.marks.asc(),df.age.desc()).show()

In [0]:
#perform some sort of aggregation after grouping the column
#df.groupBy('gender').sum("marks").show()

In [0]:
df.groupBy('gender').count().show()

+------+-----+
|gender|count|
+------+-----+
|Female|  501|
|  Male|  499|
+------+-----+



In [0]:
df.groupBy('course').count().show()

+------+-----+
|course|count|
+------+-----+
|    PF|  166|
|    DB|  157|
|   MVC|  157|
|   DSA|  176|
| Cloud|  192|
|   OOP|  152|
+------+-----+



In [0]:
#df.groupBy('gender').max('marks').show()

In [0]:
df.groupBy('course','gender').count().show()

+------+------+-----+
|course|gender|count|
+------+------+-----+
|   OOP|  Male|   70|
|    DB|  Male|   82|
| Cloud|Female|  106|
|   MVC|  Male|   86|
|   DSA|Female|   98|
|    PF|  Male|   97|
|   MVC|Female|   71|
| Cloud|  Male|   86|
|    PF|Female|   69|
|   DSA|  Male|   78|
|    DB|Female|   75|
|   OOP|Female|   82|
+------+------+-----+



In [0]:
#perform muliple aggregation function
from pyspark.sql.functions import sum, avg, max, min, mean, count
df.groupBy('course').count().show()

+------+-----+
|course|count|
+------+-----+
|    PF|  166|
|    DB|  157|
|   MVC|  157|
|   DSA|  176|
| Cloud|  192|
|   OOP|  152|
+------+-----+



In [0]:
df.groupBy('course').agg(count('*')).show()

+------+--------+
|course|count(1)|
+------+--------+
|    PF|     166|
|    DB|     157|
|   MVC|     157|
|   DSA|     176|
| Cloud|     192|
|   OOP|     152|
+------+--------+



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, udf
spark = SparkSession.builder.appName('Spark DataFrame').getOrCreate()

In [0]:
df = spark.read.options(header=True, inferSchema=True).csv('/FileStore/tables/officedata.csv')
df.show()

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|        James|     Sales|   NY| 90000| 34|10000|
|      Michael|     Sales|   NY| 86000| 56|20000|
|       Robert|     Sales|   CA| 81000| 30|23000|
|        Maria|   Finance|   CA| 90000| 24|23000|
|        Raman|   Finance|   CA| 99000| 40|24000|
|        Scott|   Finance|   NY| 83000| 36|19000|
|          Jen|   Finance|   NY| 79000| 53|15000|
|         Jeff| Marketing|   CA| 80000| 25|18000|
|        Kumar| Marketing|   NY| 91000| 50|21000|
+-------------+----------+-----+------+---+-----+



In [0]:
#want to induce a new column
from pyspark.sql.types import IntegerType, DoubleType
def total_salary(salary, bonus):
    return salary + bonus

total_salaryudf = udf(lambda x,y: total_salary(x,y), IntegerType()) #integertype-> return type of the value

df.withColumn('total_salary', total_salaryudf(df.salary, df.bonus)).show()

+-------------+----------+-----+------+---+-----+------------+
|employee_name|department|state|salary|age|bonus|total_salary|
+-------------+----------+-----+------+---+-----+------------+
|        James|     Sales|   NY| 90000| 34|10000|      100000|
|      Michael|     Sales|   NY| 86000| 56|20000|      106000|
|       Robert|     Sales|   CA| 81000| 30|23000|      104000|
|        Maria|   Finance|   CA| 90000| 24|23000|      113000|
|        Raman|   Finance|   CA| 99000| 40|24000|      123000|
|        Scott|   Finance|   NY| 83000| 36|19000|      102000|
|          Jen|   Finance|   NY| 79000| 53|15000|       94000|
|         Jeff| Marketing|   CA| 80000| 25|18000|       98000|
|        Kumar| Marketing|   NY| 91000| 50|21000|      112000|
+-------------+----------+-----+------+---+-----+------------+



In [0]:
def increment(state, salary, bonus):
    if state == 'NY':
        return 0.1*salary + 0.05*bonus
    else:
        return 0.12*salary + 0.03*bonus
    
incrementudf = udf(lambda x,y,z: increment(x,y,z), DoubleType())
df.withColumn('increment', incrementudf(df.state, df.salary, df.bonus)).show()
        

+-------------+----------+-----+------+---+-----+---------+
|employee_name|department|state|salary|age|bonus|increment|
+-------------+----------+-----+------+---+-----+---------+
|        James|     Sales|   NY| 90000| 34|10000|   9500.0|
|      Michael|     Sales|   NY| 86000| 56|20000|   9600.0|
|       Robert|     Sales|   CA| 81000| 30|23000|  10410.0|
|        Maria|   Finance|   CA| 90000| 24|23000|  11490.0|
|        Raman|   Finance|   CA| 99000| 40|24000|  12600.0|
|        Scott|   Finance|   NY| 83000| 36|19000|   9250.0|
|          Jen|   Finance|   NY| 79000| 53|15000|   8650.0|
|         Jeff| Marketing|   CA| 80000| 25|18000|  10140.0|
|        Kumar| Marketing|   NY| 91000| 50|21000|  10150.0|
+-------------+----------+-----+------+---+-----+---------+



In [0]:
df = spark.read.options(header=True, inferSchema=True).csv('/FileStore/tables/studentdata.csv')
df.groupBy('gender', 'age', 'course').count()


Out[13]: DataFrame[gender: string, age: int, course: string, count: bigint]

In [0]:
df = df.withColumn("dummy", col('age')*100)
df.show()

+---+------+----------------+------+------+-----+--------------------+-----+
|age|gender|            name|course|  roll|marks|               email|dummy|
+---+------+----------------+------+------+-----+--------------------+-----+
| 28|Female| Hubert Oliveras|    DB|  2984|   59|Annika Hoffman_Na...| 2800|
| 29|Female|Toshiko Hillyard| Cloud| 12899|   62|Margene Moores_Ma...| 2900|
| 28|  Male|  Celeste Lollis|    PF| 21267|   45|Jeannetta Golden_...| 2800|
| 29|Female|    Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...| 2900|
| 28|  Male|  Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...| 2800|
| 28|  Male|  Margene Moores|   MVC| 52771|   32|Toshiko Hillyard_...| 2800|
| 28|  Male|     Neda Briski|   OOP| 61973|   69|Alberta Freund_El...| 2800|
| 28|Female|    Claude Panos| Cloud| 72409|   85|Sheryll Towler_Al...| 2800|
| 28|  Male|  Celeste Lollis|   MVC| 81492|   64|Nicole Harwood_Cl...| 2800|
| 29|  Male|  Cordie Harnois|   OOP| 92882|   51|Judie Chipps_Clem...| 2900|

In [0]:
df.cache()

Out[16]: DataFrame[age: int, gender: string, name: string, course: string, roll: int, marks: int, email: string, dummy: int]

In [0]:
df.show()

+---+------+----------------+------+------+-----+--------------------+-----+
|age|gender|            name|course|  roll|marks|               email|dummy|
+---+------+----------------+------+------+-----+--------------------+-----+
| 28|Female| Hubert Oliveras|    DB|  2984|   59|Annika Hoffman_Na...| 2800|
| 29|Female|Toshiko Hillyard| Cloud| 12899|   62|Margene Moores_Ma...| 2900|
| 28|  Male|  Celeste Lollis|    PF| 21267|   45|Jeannetta Golden_...| 2800|
| 29|Female|    Elenore Choy|    DB| 32877|   29|Billi Clore_Mitzi...| 2900|
| 28|  Male|  Sheryll Towler|   DSA| 41487|   41|Claude Panos_Judi...| 2800|
| 28|  Male|  Margene Moores|   MVC| 52771|   32|Toshiko Hillyard_...| 2800|
| 28|  Male|     Neda Briski|   OOP| 61973|   69|Alberta Freund_El...| 2800|
| 28|Female|    Claude Panos| Cloud| 72409|   85|Sheryll Towler_Al...| 2800|
| 28|  Male|  Celeste Lollis|   MVC| 81492|   64|Nicole Harwood_Cl...| 2800|
| 29|  Male|  Cordie Harnois|   OOP| 92882|   51|Judie Chipps_Clem...| 2900|